<a href="https://colab.research.google.com/github/RohanSai22/GithubRepo/blob/main/All_Frames_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install open3d

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.7/399.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
import cv2
import open3d as o3d
import numpy as np
import os

def extract_walls(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise and improve edge detection
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detector to find edges in the image
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a mask to represent walls
    wall_mask = np.zeros_like(gray)

    # Iterate through contours
    for contour in contours:
        # Calculate the area of the contour
        area = cv2.contourArea(contour)

        # Filter out small contours (noise)
        if area > 1000:
            # Draw the contour on the mask
            cv2.drawContours(wall_mask, [contour], -1, (255), thickness=cv2.FILLED)

    # Use the mask to extract walls from the original image
    walls = cv2.bitwise_and(image, image, mask=wall_mask)

    return walls

def process_video(video_path, output_folder):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Create Open3D point cloud
    point_cloud = o3d.geometry.PointCloud()

    for i in range(frame_count):
        ret, frame = cap.read()

        # Extract walls from the frame
        walls = extract_walls(frame)

        # Convert to Open3D format
        colors = walls.reshape((-1, 3)).astype(np.float64) / 255.0
        points = np.argwhere(walls == 0).astype(np.float64)
        points[:, [0, 1]] = points[:, [1, 0]]  # Swap x and y

        print(f"Frame {i + 1}: Detected {len(points)} points")

        if len(points) > 0:
            point_cloud.points.extend(o3d.utility.Vector3dVector(points.tolist()))
            point_cloud.colors.extend(o3d.utility.Vector3dVector(colors.tolist()))

    cap.release()

    if len(point_cloud.points) > 0:
        # Save point cloud to file
        o3d.io.write_point_cloud(os.path.join(output_folder, "room_model.ply"), point_cloud)
        print("Point cloud saved successfully.")
    else:
        print("No points detected. Point cloud not saved.")

def process_images(images_folder, output_folder):
    # Create Open3D point cloud
    point_cloud = o3d.geometry.PointCloud()

    for filename in os.listdir(images_folder):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            image_path = os.path.join(images_folder, filename)

            # Read the image
            frame = cv2.imread(image_path)

            # Extract walls from the frame
            walls = extract_walls(frame)

            # Convert to Open3D format
            colors = walls.reshape((-1, 3)).astype(np.float64) / 255.0
            points = np.argwhere(walls == 0).astype(np.float64)
            points[:, [0, 1]] = points[:, [1, 0]]  # Swap x and y

            print(f"Image {filename}: Detected {len(points)} points")

            if len(points) > 0:
                point_cloud.points.extend(o3d.utility.Vector3dVector(points.tolist()))
                point_cloud.colors.extend(o3d.utility.Vector3dVector(colors.tolist()))

    if len(point_cloud.points) > 0:
        # Save point cloud to file
        o3d.io.write_point_cloud(os.path.join(output_folder, "room_model.ply"), point_cloud)
        print("Point cloud saved successfully.")
    else:
        print("No points detected. Point cloud not saved.")

if __name__ == "__main__":
    input_type = input("Enter 'video' or 'images': ").lower()

    if input_type == "video":
        video_path = input("Enter the path to the video file: ")
        output_folder = input("Enter the output folder: ")
        os.makedirs(output_folder, exist_ok=True)
        process_video(video_path, output_folder)
    elif input_type == "images":
        images_folder = input("Enter the path to the folder containing images: ")
        output_folder = input("Enter the output folder: ")
        os.makedirs(output_folder, exist_ok=True)
        process_images(images_folder, output_folder)
    else:
        print("Invalid input. Please enter 'video' or 'images'.")

Enter 'video' or 'images': video
Enter the path to the video file: /content/sample_data/videos/production_id_3770033 (1080p).mp4
Enter the output folder: /content/sample_data/otput
Frame 1: Detected 6126045 points
Frame 2: Detected 6170022 points
Frame 3: Detected 6136086 points
Frame 4: Detected 6182406 points
Frame 5: Detected 6132393 points
Frame 6: Detected 6120192 points
Frame 7: Detected 6192807 points
Frame 8: Detected 6138363 points
Frame 9: Detected 6138120 points
Frame 10: Detected 6183339 points
Frame 11: Detected 6184791 points
Frame 12: Detected 6121893 points
Frame 13: Detected 6124842 points
Frame 14: Detected 6181230 points
Frame 15: Detected 6182286 points
Frame 16: Detected 6128823 points
Frame 17: Detected 6122601 points
Frame 18: Detected 6122961 points
Frame 19: Detected 6126426 points
Frame 20: Detected 6125967 points
Frame 21: Detected 6117123 points
Frame 22: Detected 6180453 points
Frame 23: Detected 6116559 points
Frame 24: Detected 6118092 points
Frame 25: De

SyntaxError: unterminated string literal (detected at line 112) (<ipython-input-1-831e4c892263>, line 112)